# GUI (Graphical User Interface)

In [1]:
import os
import cv2
import tkinter as tk
from tkinter import simpledialog, filedialog, messagebox
import torchvision
import torchvision.transforms as T
from PIL import Image, ImageTk
from threading import Thread
import torch
from ultralytics import YOLO

def play_video(video_path, canvas, label, tracker=None, detections=None, model_type=None):
    cap = cv2.VideoCapture(video_path)
    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if tracker:
            success, bbox = tracker.update(frame)
            if success:
                x, y, w, h = map(int, bbox)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                cv2.putText(frame, "Tracking", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        elif detections and model_type == "mask_rcnn":
            for i, (box, score) in enumerate(detections):
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 165, 0), 2)
                text = f"ID:{i} (Conf: {score:.2f})"
                cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 165, 0), 2)

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (canvas_width, canvas_height))
        frame_image = ImageTk.PhotoImage(Image.fromarray(frame))

        canvas.create_image(0, 0, anchor=tk.NW, image=frame_image)
        label.image = frame_image
        label.update()

    cap.release()


def play_multi_video(video_path, canvas, label, trackers=None):
    cap = cv2.VideoCapture(video_path)
    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if trackers:
            for tracker in trackers:
                success, bbox = tracker.update(frame)
                if success:
                    x, y, w, h = map(int, bbox)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (canvas_width, canvas_height))
        frame_image = ImageTk.PhotoImage(Image.fromarray(frame))

        canvas.create_image(0, 0, anchor=tk.NW, image=frame_image)
        label.image = frame_image
        label.update()

    cap.release()


# Function to update the detection frame based on the checkbox state
def update_detection_frame(frame, detections, model_type):
    for i, detection in enumerate(detections):
        if model_type == "mask_rcnn":
            # Mask R-CNN detections: (box, score)
            box, score = detection
            x1, y1, x2, y2 = map(int, box)  # Convert to integers
        elif model_type == "yolov5":
            # YOLOv5 detections: [x1, y1, x2, y2, conf, cls]
            x1, y1, x2, y2, conf, _ = map(float, detection[:6])  # Ensure all values are floats
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])  # Convert to integers
            score = conf  # Use confidence score as score
        elif model_type == "yolov8":
            # YOLOv8 detections: (box, conf, cls)
            box, conf, _ = detection
            x1, y1, x2, y2 = map(int, box)  # Convert to integers
            score = conf  # Confidence score

        # Draw the bounding box
        if model_type == "mask_rcnn":
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 2)  # Orange box
        elif model_type == "yolov5":
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 0), 2)  # Orange box
        elif model_type == "yolov8":
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Orange box


        # Add the text (ID and optionally confidence score)
        if model_type == "mask_rcnn":
            text = f"ID:{i}" if not show_confidence_var.get() else f"ID:{i} (Conf: {score:.2f})"
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 255), 2)
        elif model_type == "yolov5":
            text = f"ID:{i}" if not show_confidence_var.get() else f"ID:{i} (Conf: {score:.2f})"
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
        elif model_type == "yolov8":
            text = f"ID:{i}" if not show_confidence_var.get() else f"ID:{i} (Conf: {score:.2f})"
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    return frame

# confidence score search box 
def validate_confidence_input(P):
    """Validate the input to ensure it's a number between 0-100"""
    if P == "":  # Allow empty field
        return True
    try:
        value = float(P)
        return 0 <= value <= 100
    except ValueError:
        return False

def on_confidence_change(event=None):
    """Handle confidence score changes"""
    try:
        value = float(confidence_entry.get())
        if 0 <= value <= 100:
            # Here you can add what happens when a valid confidence score is entered
            print(f"Confidence score set to: {value}%")
            # confidence_label.config(text=f"Current confidence: {value}%")
        else:
            messagebox.showwarning("Invalid Input", "Please enter a number between 0 and 100")
            confidence_entry.delete(0, tk.END)
    except ValueError:
        if confidence_entry.get() != "":  # Only show error if field isn't empty
            messagebox.showwarning("Invalid Input", "Please enter a valid number")
            confidence_entry.delete(0, tk.END)

#mask_rcnn
def process_first_frame_mask_rcnn(video_path, detection_canvas, detection_label):
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    if not ret:
        messagebox.showerror("Error", "Failed to read video.")
        cap.release()
        return

    transform = T.Compose([T.ToTensor()])
    input_frame = transform(frame).unsqueeze(0)

    # Get the user-defined confidence threshold
    confidence_threshold = get_confidence_threshold()  # This gets the value from the input box

    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()

    with torch.no_grad():
        predictions = model(input_frame)

    pred_boxes = predictions[0]["boxes"].cpu().numpy()
    pred_scores = predictions[0]["scores"].cpu().numpy()

    # Use the dynamic confidence threshold
    filtered_detections = [
        (box, score) for box, score in zip(pred_boxes, pred_scores) if score > confidence_threshold
    ]

    frame = update_detection_frame(frame, filtered_detections, "mask_rcnn")

    detected_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    detected_frame = cv2.resize(detected_frame, (detection_canvas.winfo_width(), detection_canvas.winfo_height()))
    detected_frame = Image.fromarray(detected_frame)
    detected_frame = ImageTk.PhotoImage(detected_frame)

    detection_canvas.create_image(0, 0, anchor=tk.NW, image=detected_frame)
    detection_label.image = detected_frame

    cap.release()
    return filtered_detections

#yolo5 
def process_first_frame_yolov5(video_path, detection_canvas, detection_label):
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    if not ret:
        messagebox.showerror("Error", "Failed to read video.")
        cap.release()
        return

    # Get the user-defined confidence threshold
    confidence_threshold = get_confidence_threshold()

    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    results = model(frame)
    detections = results.pred[0]

    # Use the dynamic confidence threshold
    filtered_detections = [d for d in detections if d[4] > confidence_threshold]

    frame = update_detection_frame(frame, filtered_detections, "yolov5")

    detected_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    detected_frame = cv2.resize(detected_frame, (detection_canvas.winfo_width(), detection_canvas.winfo_height()))
    detected_frame = Image.fromarray(detected_frame)
    detected_frame = ImageTk.PhotoImage(detected_frame)

    detection_canvas.create_image(0, 0, anchor=tk.NW, image=detected_frame)
    detection_label.image = detected_frame

    cap.release()
    return filtered_detections

#yolo8
def process_first_frame_yolov8(video_path, detection_canvas, detection_label):
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    if not ret:
        messagebox.showerror("Error", "Failed to read video.")
        cap.release()
        return

    # Get the user-defined confidence threshold
    confidence_threshold = get_confidence_threshold()

    model = YOLO('yolov8s.pt')
    results = model(frame)
    detections = results[0].boxes
    boxes = detections.xyxy.cpu().numpy()
    confs = detections.conf.cpu().numpy()
    classes = detections.cls.cpu().numpy()

    # Use the dynamic confidence threshold
    filtered_detections = [(box, conf, cls) for box, conf, cls in zip(boxes, confs, classes) 
                          if conf > confidence_threshold]

    frame = update_detection_frame(frame, filtered_detections, "yolov8")

    detected_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    detected_frame = cv2.resize(detected_frame, (detection_canvas.winfo_width(), detection_canvas.winfo_height()))
    detected_frame = Image.fromarray(detected_frame)
    detected_frame = ImageTk.PhotoImage(detected_frame)

    detection_canvas.create_image(0, 0, anchor=tk.NW, image=detected_frame)
    detection_label.image = detected_frame

    cap.release()
    return filtered_detections

def upload_video():
    file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4 *.avi *.mov")])
    if file_path:
        input_path_var.set(file_path)
        display_video(file_path, input_canvas, input_label)


# Mask R-CNN button function
def process_with_mask_rcnn():
    input_path = input_path_var.get()
    if not input_path:
        messagebox.showerror("Error", "Please upload a video first.")
        return

    # Get current confidence threshold from input
    confidence_threshold = get_confidence_threshold()
    
    if confidence_threshold is None:
        messagebox.showerror("Error", "Please enter a valid confidence threshold (0-100%).")
        return

    try:
        global detections, current_model
        detections = process_first_frame_mask_rcnn(input_path, detection_canvas, detection_label)
        current_model = "mask_rcnn"
        
        # Update the UI to show current confidence threshold being used
        status_message = f"Mask R-CNN processing complete (Confidence Threshold: {confidence_threshold:.1%})"
        # Assuming you have a status label in your UI
        if 'status_label' in globals():
            status_label.config(text=status_message)
            
        return detections
    except Exception as e:
        messagebox.showerror("Error", f"Error processing video: {str(e)}")
        return None

# Add this function to update detections when confidence threshold changes
def update_mask_rcnn_detections():
    if current_model == "mask_rcnn" and detections:
        new_confidence = get_confidence_threshold()
        # Reprocess the current frame with new confidence threshold
        process_with_mask_rcnn()

# yolo5
def process_with_yolov5():
    input_path = input_path_var.get()
    if not input_path:
        messagebox.showerror("Error", "Please upload a video first.")
        return

    # Get current confidence threshold from input
    confidence_threshold = get_confidence_threshold()
    
    if confidence_threshold is None:
        messagebox.showerror("Error", "Please enter a valid confidence threshold (0-100%).")
        return

    try:
        global detections, current_model
        detections = process_first_frame_yolov5(input_path, detection_canvas, detection_label)
        current_model = "yolov5"
        
        # Update the UI to show current confidence threshold being used
        status_message = f"YOLOv5 processing complete (Confidence Threshold: {confidence_threshold:.1%})"
        # Assuming you have a status label in your UI
        if 'status_label' in globals():
            status_label.config(text=status_message)
            
        return detections
    except Exception as e:
        messagebox.showerror("Error", f"Error processing video: {str(e)}")
        return None

def update_yolov5_detections():
    if current_model == "yolov5" and detections:
        new_confidence = get_confidence_threshold()
        # Reprocess the current frame with new confidence threshold
        process_with_yolov5()

# Update the apply button command to handle both models
def update_detections():
    if current_model == "yolov5":
        update_yolov5_detections()
    elif current_model == "mask_rcnn":
        update_mask_rcnn_detections()

# yolo8
def process_with_yolov8():
    input_path = input_path_var.get()
    if not input_path:
        messagebox.showerror("Error", "Please upload a video first.")
        return

    # Get current confidence threshold from input
    confidence_threshold = get_confidence_threshold()
    
    if confidence_threshold is None:
        messagebox.showerror("Error", "Please enter a valid confidence threshold (0-100%).")
        return

    try:
        global detections, current_model
        detections = process_first_frame_yolov8(input_path, detection_canvas, detection_label)
        current_model = "yolov8"
        
        # Update the UI to show current confidence threshold being used
        status_message = f"YOLOv8 processing complete (Confidence Threshold: {confidence_threshold:.1%})"
        # Assuming you have a status label in your UI
        if 'status_label' in globals():
            status_label.config(text=status_message)
            
        return detections
    except Exception as e:
        messagebox.showerror("Error", f"Error processing video: {str(e)}")
        return None

def update_yolov8_detections():
    if current_model == "yolov8" and detections:
        new_confidence = get_confidence_threshold()
        # Reprocess the current frame with new confidence threshold
        process_with_yolov8()

#track object def
def track_object():
    input_path = input_path_var.get()
    if not input_path:
        messagebox.showerror("Error", "Please upload a video and process it first.")
        return

    if detections is None or len(detections) == 0:
        messagebox.showerror("Error", "No detections to track. Run detection first.")
        return

    # Get current confidence threshold
    confidence_threshold = get_confidence_threshold()

    # Filter detections based on current model and confidence threshold
    if current_model == "yolov5":
        valid_detections = [d for d in detections if d[4] > confidence_threshold]
        confidence_scores = [d[4] for d in valid_detections]
    elif current_model == "yolov8":
        valid_detections = [(box, conf, cls) for box, conf, cls in detections if conf > confidence_threshold]
        confidence_scores = [conf for _, conf, _ in valid_detections]
    elif current_model == "mask_rcnn":
        valid_detections = [(box, score) for box, score in detections if score > confidence_threshold]
        confidence_scores = [score for _, score in valid_detections]

    if not valid_detections:
        messagebox.showerror("Error", f"No valid detections above confidence threshold of {confidence_threshold:.1%}")
        return

    # Sort detections by confidence score in descending order to maintain consistent ordering
    valid_detections = [x for _, x in sorted(zip(confidence_scores, valid_detections), reverse=True)]

    cap = cv2.VideoCapture(input_path)
    ret, frame = cap.read()
    if not ret:
        messagebox.showerror("Error", "Failed to read video.")
        cap.release()
        return

    # Display confidence scores for each object
    confidence_info = "\n".join([f"Object {i}: {confidence_scores[i]:.1%}" 
                                for i in range(len(valid_detections))])
    
    object_id = simpledialog.askinteger("Select Object", 
                                      f"Enter object ID (0 to {len(valid_detections) - 1})\n\n"
                                      f"Detected objects (sorted by confidence):\n{confidence_info}")

    if  object_id < 0 or object_id >= len(valid_detections):
        messagebox.showerror("Error", "Invalid object ID selected.")
        cap.release()
        return

    # Extract bounding box based on model type
    if current_model == "yolov5":
        x1, y1, x2, y2, _, _ = valid_detections[object_id].tolist()
    elif current_model == "yolov8":
        box, _, _ = valid_detections[object_id]
        x1, y1, x2, y2 = map(int, box)
    elif current_model == "mask_rcnn":
        box, _ = valid_detections[object_id]
        x1, y1, x2, y2 = map(int, box)

    bbox = (int(x1), int(y1), int(x2 - x1), int(y2 - y1))
    tracker = cv2.TrackerCSRT_create()
    tracker.init(frame, bbox)

    display_video(input_path, output_canvas, output_label, tracker)
    cap.release()

def display_video(video_path, canvas, label, tracker=None):
    thread = Thread(target=play_video, args=(video_path, canvas, label, tracker))
    thread.daemon = True
    thread.start()

def track_multi_objects():
    input_path = input_path_var.get()
    if not input_path:
        messagebox.showerror("Error", "Please upload a video and process it first.")
        return

    if detections is None or len(detections) == 0:
        messagebox.showerror("Error", "No detections to track. Run detection first.")
        return

    # Get current confidence threshold
    confidence_threshold = get_confidence_threshold()

    # Filter detections based on the confidence threshold and model type
    if current_model == "yolov5":
        valid_detections = [d for d in detections if d[4] > confidence_threshold]
        confidence_scores = [d[4] for d in valid_detections]
    elif current_model == "yolov8":
        valid_detections = [(box, conf, cls) for box, conf, cls in detections if conf > confidence_threshold]
        confidence_scores = [conf for _, conf, _ in valid_detections]
    elif current_model == "mask_rcnn":
        valid_detections = [(box, score) for box, score in detections if score > confidence_threshold]
        confidence_scores = [score for _, score in valid_detections]

    if not valid_detections:
        messagebox.showerror("Error", f"No valid detections above confidence threshold of {confidence_threshold:.1%}")
        return

    # Sort detections by confidence score in descending order
    valid_detections = [x for _, x in sorted(zip(confidence_scores, valid_detections), reverse=True)]

    cap = cv2.VideoCapture(input_path)
    ret, frame = cap.read()
    if not ret:
        messagebox.showerror("Error", "Failed to read video.")
        cap.release()
        return

    # Display detected objects with confidence scores
    confidence_info = "\n".join([f"Object {i}: {confidence_scores[i]:.1%}" for i in range(len(valid_detections))])
    object_ids = simpledialog.askstring("Select Objects", 
                                        f"Enter object IDs separated by commas (or 'all' for tracking all objects):\n\n"
                                        f"Detected objects (sorted by confidence):\n{confidence_info}")

    if object_ids is None:
        cap.release()
        return

    object_ids = object_ids.strip().lower()
    if object_ids == 'all':
        selected_indices = range(len(valid_detections))
    else:
        try:
            selected_indices = [int(i) for i in object_ids.split(',') if 0 <= int(i) < len(valid_detections)]
        except ValueError:
            messagebox.showerror("Error", "Invalid object IDs entered.")
            cap.release()
            return

    trackers = []
    for i in selected_indices:
        if current_model == "yolov5":
            x1, y1, x2, y2, _, _ = valid_detections[i].tolist()
        elif current_model == "yolov8":
            box, _, _ = valid_detections[i]
            x1, y1, x2, y2 = map(int, box)
        elif current_model == "mask_rcnn":
            box, _ = valid_detections[i]
            x1, y1, x2, y2 = map(int, box)

        bbox = (int(x1), int(y1), int(x2 - x1), int(y2 - y1))
        tracker = cv2.TrackerCSRT_create()
        tracker.init(frame, bbox)
        trackers.append(tracker)

    display_multi_video(input_path, output_canvas, output_label, trackers)
    cap.release()


def display_multi_video(video_path, canvas, label, tracker=None):
    thread = Thread(target=play_multi_video, args=(video_path, canvas, label, tracker))
    thread.daemon = True
    thread.start()

# Initialize GUI
root = tk.Tk()
root.title("Object Detection Tool")
# root.geometry("2000x2000")
def toggle_fullscreen(event):
    root.attributes('-fullscreen', not root.attributes('-fullscreen'))

root.bind('<Escape>', toggle_fullscreen)

# Add a variable to store the state of the checkbox
show_confidence_var = tk.BooleanVar(value=False)  # Default to not showing confidence scores

input_path_var = tk.StringVar()
detections = None
current_model = None

# Welcome label
welcome_label = tk.Label(root, text="Welcome to Object Detection Tool!", font=("Helvetica", 30))
welcome_label.pack(pady=20)

# Upload button
upload_button = tk.Button(root, text="Upload Video", font=("Helvetica", 27), command=upload_video, fg="red")
upload_button.pack(pady=20)

# Video display frames
frame = tk.Frame(root)
frame.pack(pady=20)

input_label = tk.Label(frame, text="Input Video", font=("Helvetica", 25))
input_label.grid(row=0, column=0, padx=20)

detection_label = tk.Label(frame, text="Detection Candidates", font=("Helvetica", 25))
detection_label.grid(row=0, column=1, padx=20)

output_label = tk.Label(frame, text="Output Video", font=("Helvetica", 25))
output_label.grid(row=0, column=2, padx=20)

input_canvas = tk.Canvas(frame, width=460, height=270, bg="black")
input_canvas.grid(row=1, column=0, padx=20, pady=20)

detection_canvas = tk.Canvas(frame, width=460, height=270, bg="black")
detection_canvas.grid(row=1, column=1, padx=20, pady=20)

output_canvas = tk.Canvas(frame, width=460, height=270, bg="black")
output_canvas.grid(row=1, column=2, padx=20, pady=20)

# YOLO buttons frame
yolo_buttons_frame = tk.Frame(frame)
yolo_buttons_frame.grid(row=2, column=0, pady=10)

# Two YOLO buttons stacked vertically
yolov5_button = tk.Button(yolo_buttons_frame, text="Run YOLOv5", font=("Helvetica", 18), command=process_with_yolov5, fg="red")
yolov5_button.pack(pady=5)

yolov8_button = tk.Button(yolo_buttons_frame, text="Run YOLOv8", font=("Helvetica", 18), command=process_with_yolov8, fg="red")
yolov8_button.pack(pady=5)

# Add Mask R-CNN button
mask_rcnn_button = tk.Button(yolo_buttons_frame, text="Run Mask R-CNN", font=("Helvetica", 18), command=process_with_mask_rcnn, fg="red")
mask_rcnn_button.pack(pady=5)

# checkbox
confidence_checkbox = tk.Checkbutton(frame, text="Show Confidence Scores", font=("Helvetica", 14), variable=show_confidence_var, command=lambda: update_detection_frame(None, detections, current_model) if detections else None)
confidence_checkbox.grid(row=2, column=1, pady=10)
confidence_checkbox.grid(row=2, column=1, sticky="n", pady=10)


## cofidence search box

# Create confidence score frame
confidence_frame = tk.Frame(frame)
confidence_frame.grid(row=3, column=0, pady=10)

# Create and pack the label
confidence_label = tk.Label(
    confidence_frame,
    text="Enter confidence threshold (0-100%):",
    font=("Helvetica", 14)
)
confidence_label.pack(side=tk.LEFT)

# Register the validation command
vcmd = (confidence_frame.register(validate_confidence_input), '%P')

# Create and pack the entry widget
confidence_entry = tk.Entry(
    confidence_frame,
    font=("Helvetica", 14),
    width=3,
    validate='key',
    validatecommand=vcmd
)
confidence_entry.pack(side=tk.LEFT, padx=(5, 10))  # Add some horizontal padding
confidence_entry.insert(0, "70")  # Default value

# Create apply button
apply_button = tk.Button(
    confidence_frame,
    text="Apply",
    font=("Helvetica", 14),
    command=on_confidence_change,
    fg="red"
)
apply_button.pack(side=tk.LEFT)

# Bind Enter key to the entry widget
confidence_entry.bind('<Return>', on_confidence_change)

# If you want to place it at specific coordinates
confidence_frame.grid(row=2, column=1, pady=10)

# Function to get the current confidence value (can be called from other parts of your code)
def get_confidence_threshold():
    try:
        return float(confidence_entry.get()) / 100.0  # Convert to 0-1 range
    except ValueError:
        return 0.5  # Default value if invalid

# Track object button
track_button = tk.Button(frame, text="Track Object", font=("Helvetica", 18), command=track_object, fg="red")
track_button.grid(row=2, column=2, sticky="n", pady=10)

# Track multi objects button
track_button = tk.Button(frame, text="Track Multi Objects", font=("Helvetica", 18), command=track_multi_objects, fg="red")
track_button.grid(row=2, column=2, pady=10)

# Run the GUI
root.mainloop()